<a href="https://colab.research.google.com/github/drshyamsundaram/pet/blob/main/Dp_xgBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Differentially Private XGBoost Implementation

This module provides a differentially private version of XGBoost for binary classification tasks.
It implements the gradient-based approach to differential privacy in machine learning,
as described in the following papers:

[1] Abadi, M., Chu, A., Goodfellow, I., McMahan, H. B., Mironov, I., Talwar, K., & Zhang, L. (2016).
    Deep learning with differential privacy. In Proceedings of the 2016 ACM SIGSAC Conference on
    Computer and Communications Security (pp. 308-318).

[2] Dwork, C., & Roth, A. (2014). The algorithmic foundations of differential privacy.
    Foundations and Trends in Theoretical Computer Science, 9(3-4), 211-407.

[3] Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System.
    In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery
    and Data Mining (pp. 785-794).

Author: [Shyam Sundaram]
Date: [25 Nov 24 ]
Version: 1.0
"""


'\nDifferentially Private XGBoost Implementation\n\nThis module provides a differentially private version of XGBoost for binary classification tasks.\nIt implements the gradient-based approach to differential privacy in machine learning,\nas described in the following papers:\n\n[1] Abadi, M., Chu, A., Goodfellow, I., McMahan, H. B., Mironov, I., Talwar, K., & Zhang, L. (2016).\n    Deep learning with differential privacy. In Proceedings of the 2016 ACM SIGSAC Conference on\n    Computer and Communications Security (pp. 308-318).\n\n[2] Dwork, C., & Roth, A. (2014). The algorithmic foundations of differential privacy.\n    Foundations and Trends in Theoretical Computer Science, 9(3-4), 211-407.\n\n[3] Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System.\n    In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery\n    and Data Mining (pp. 785-794).\n\nAuthor: [Shyam Sundaram]\nDate: [25 Nov 24 ]\nVersion: 1.0\n'

In [5]:

import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy.special import expit

class DPXGBoost:
    def __init__(self, epsilon, delta, num_trees, max_depth, learning_rate):
        """
        Initialize the Differentially Private XGBoost model.

        Args:
            epsilon (float): Privacy parameter that controls the privacy budget.
            delta (float): Probability of privacy violation.
            num_trees (int): Number of trees in the ensemble.
            max_depth (int): Maximum depth of each tree.
            learning_rate (float): Step size shrinkage used to prevent overfitting.

        The epsilon and delta parameters together define the privacy guarantee.
        Smaller values provide stronger privacy but may reduce model accuracy.
        """
        self.epsilon = epsilon
        self.delta = delta
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.models = []

    def _add_noise(self, gradients, hessians):
        """
        Add Laplace noise to gradients and hessians to achieve differential privacy.

        Args:
            gradients (np.array): Computed gradients.
            hessians (np.array): Computed hessians.

        Returns:
            tuple: Noisy gradients and hessians.

        This method implements the Laplace mechanism as described in [2].
        The noise scale is calculated based on the sensitivity of the gradients
        and the privacy parameters (epsilon and delta).
        """
        sensitivity = 2  # Assuming binary classification with log loss
        noise_scale = sensitivity * np.sqrt(2 * np.log(1.25 / self.delta)) / self.epsilon
        noisy_gradients = gradients + np.random.laplace(0, noise_scale, size=gradients.shape)
        noisy_hessians = hessians + np.random.laplace(0, noise_scale, size=hessians.shape)
        return noisy_gradients, noisy_hessians

    def fit(self, X, y):
        """
        Fit the Differentially Private XGBoost model to the training data.

        Args:
            X (np.array): Training features.
            y (np.array): Training labels.

        This method implements the differentially private stochastic gradient descent
        algorithm as described in [1], adapted for XGBoost. It iteratively builds
        trees using noisy gradients and hessians.
        """
        for _ in range(self.num_trees):
            gradients = -y + expit(self.predict(X))
            hessians = expit(self.predict(X)) * (1 - expit(self.predict(X)))

            noisy_gradients, noisy_hessians = self._add_noise(gradients, hessians)

            dtrain = xgb.DMatrix(X, label=y)
            dtrain.set_base_margin(self.predict(X))

            params = {
                'objective': 'binary:logistic',
                'max_depth': self.max_depth,
                'learning_rate': self.learning_rate,
                'silent': 1
            }

            model = xgb.train(params, dtrain, num_boost_round=1,
                              obj=lambda _, pred: (noisy_gradients, noisy_hessians))
            self.models.append(model)

    def predict(self, X):
        """
        Make predictions using the trained Differentially Private XGBoost model.

        Args:
            X (np.array): Features to predict.

        Returns:
            np.array: Predicted probabilities.

        This method aggregates predictions from all trees in the ensemble,
        as described in [3], but with the differentially private trees trained
        in the fit method.
        """
        if not self.models:
            return np.zeros(X.shape[0])

        dtest = xgb.DMatrix(X)
        predictions = sum(model.predict(dtest) for model in self.models)
        return predictions



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:30:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [6]:
# Usage example
X, y = np.random.rand(1000, 10), np.random.randint(0, 2, 1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dp_xgb = DPXGBoost(epsilon=0.1, delta=1e-5, num_trees=10, max_depth=3, learning_rate=0.1)
dp_xgb.fit(X_train, y_train)

predictions = dp_xgb.predict(X_test)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:30:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:30:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [10]:
# prompt: Create a UI for the configuration for the code with model execute button

import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy.special import expit

# ... (Your DPXGBoost class code from the previous response) ...

# UI elements
epsilon_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Epsilon:')
delta_slider = widgets.FloatLogSlider(value=1e-5, min=-10, max=-1, step=0.1, description='Delta:')
num_trees_slider = widgets.IntSlider(value=10, min=1, max=100, step=1, description='Num Trees:')
max_depth_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description='Max Depth:')
learning_rate_slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Learning Rate:')
execute_button = widgets.Button(description='Execute Model')
output_area = widgets.Output()

# Layout
input_widgets = widgets.VBox([epsilon_slider, delta_slider, num_trees_slider, max_depth_slider, learning_rate_slider, execute_button])
display(widgets.HBox([input_widgets, output_area]))

# Execute button click handler
def on_button_clicked(b):
    with output_area:
        clear_output()  # Clear previous output
        try:
            # Get parameter values from the UI
            epsilon = epsilon_slider.value
            delta = delta_slider.value
            num_trees = num_trees_slider.value
            max_depth = max_depth_slider.value
            learning_rate = learning_rate_slider.value

            # Example data (replace with your actual data)
            X, y = np.random.rand(1000, 10), np.random.randint(0, 2, 1000)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Create and train the model
            dp_xgb = DPXGBoost(epsilon, delta, num_trees, max_depth, learning_rate)
            dp_xgb.fit(X_train, y_train)
            predictions = dp_xgb.predict(X_test)

            print("Model execution complete.")
            print("Predictions:", predictions) # Display some output

        except Exception as e:
            print(f"An error occurred: {e}")

execute_button.on_click(on_button_clicked)

In [9]:
dp_xgb = DPXGBoost(config)

TypeError: DPXGBoost.__init__() missing 4 required positional arguments: 'delta', 'num_trees', 'max_depth', and 'learning_rate'